# Advanced Lane Finding Project

---

**Author:** Sergey Morozov

---

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# set up logging
import logging
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s] %(message)s")

# constants
import numpy as np
IMAGE_SIZE=(720, 1280)
CHESSBOARD_IMAGE_DIR = "chessboard_images"
TEST_IMAGE_DIR = "test_images"
OUTPUT_IMAGE_DIR = "output_images"
OUTPUT_VIDEO_DIR = "output_videos"
ABSOLUTE_SOBEL_X = (7, 20, 100)
ABSOLUTE_SOBEL_Y = (7, 20, 100)
MAGNITUDE_SOBEL = (3, 30, 100)
DIRECTION_SOBEL = (31, 0.5, 1.0)
S_CHANNEL_THRESHOLD = (170, 255)
WARP_SRC = np.float32([(532, 496),
                       (756, 496),
                       (288, 664),
                       (1016, 664)])
WARP_DST = np.float32([(WARP_SRC[2][0], WARP_SRC[2][1] - 200),
                       (WARP_SRC[3][0], WARP_SRC[3][1] - 200),
                       (WARP_SRC[2][0], WARP_SRC[2][1]),
                       (WARP_SRC[3][0], WARP_SRC[3][1])])
SLIDING_WINDOW_PARAMS = (9, 100, 50)
REGION_OF_INTEREST_VERTS = np.array([[
            (0, IMAGE_SIZE[0]),
            (IMAGE_SIZE[1] / 2, IMAGE_SIZE[0] / 2 + 45),
            (IMAGE_SIZE[1] / 2, IMAGE_SIZE[0] / 2 + 45),
            (IMAGE_SIZE[1],     IMAGE_SIZE[0])
        ]], dtype = np.int32)

# all lane finding code is in advanced_lane_finding.py
from advanced_lane_finding import *

# initialize class instance containing advanced lane line detection methods
lane_finder = AdvancedLaneFinder(
    image_size=IMAGE_SIZE,
    chessboard_image_dir=CHESSBOARD_IMAGE_DIR,
    absolute_sobel_x=ABSOLUTE_SOBEL_X,
    absolute_sobel_y=ABSOLUTE_SOBEL_Y,
    magnitude_sobel=MAGNITUDE_SOBEL,
    direction_sobel=DIRECTION_SOBEL,
    s_channel_thresh=S_CHANNEL_THRESHOLD,
    warp_perspective=(WARP_SRC, WARP_DST),
    sliding_window_params=SLIDING_WINDOW_PARAMS,
    region_of_interest_verts=REGION_OF_INTEREST_VERTS,
)

## Compute Camera Calibration Matrix and Distortion Coefficients

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# camera calibrarion will happen once inside the pipeline

# undistort chessboard image calibration2.jpg;
# visially it is the mostly distorted image
test_img = cv2.imread(os.path.join(CHESSBOARD_IMAGE_DIR, "calibration2.jpg"))

# undistort test chessboard image
test_img_undistorted = lane_finder.pipeline(image=test_img, stop_on_step='distortion_correction')

# visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(test_img_undistorted)
ax2.set_title('Undistorted Image', fontsize=30)

Note that in the initial set of chessboard images, provided by Udacity, image *calibration7.jpg* and *calibration15.jpg* had shape (1281, 721) while all other images had shape (1280, 720). These discrepancy led to inaccrate calculations. Images with incorrect shape (*calibration7.jpg* and *calibration15.jpg*) were cropped by 1 pixel and in this repository all images have (1280, 720) shape.

## Apply Distortion Correction to Raw Images

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline

# for each test image apply pipeline steps and stop after undistortion applied
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_undist = lane_finder.pipeline(image=img, stop_on_step='distortion_correction')
    
    # create output directory for images, if does not exist
    if not os.path.isdir(OUTPUT_IMAGE_DIR):
        os.mkdir(OUTPUT_IMAGE_DIR)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_undistorted.jpg"), img_undist)
    
# visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test1.jpg"))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)

img_undist = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test1_undistorted.jpg"))
ax2.imshow(img_undist)
ax2.set_title('Undistorted Image', fontsize=30)

## Create Thresholded Binary Images

In [ ]:
import cv2 
import os
import matplotlib.pyplot as plt
%matplotlib inline

# for each test image apply pipeline steps and stop after thresholds applied
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_thresh = lane_finder.pipeline(image=img, stop_on_step='apply_thresholds')
    
    # create output directory for images, if does not exist
    if not os.path.isdir(OUTPUT_IMAGE_DIR):
        os.mkdir(OUTPUT_IMAGE_DIR)
        
    # save thresholded image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_thresholded.jpg"), img_thresh)

# visualize thresholded images (use difficult cases)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test5.jpg"))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)

img_thresh = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test5_thresholded.jpg"))
ax2.imshow(img_thresh, cmap='gray')
ax2.set_title('Thresholded Image', fontsize=30)

## Region of Interest

In [ ]:
import cv2 
import os
import matplotlib.pyplot as plt
%matplotlib inline

# for each test image apply pipeline steps and stop after region of interest applied
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_reg_of_int = lane_finder.pipeline(image=img, stop_on_step='region_of_interest')
    
    # create output directory for images, if does not exist
    if not os.path.isdir(OUTPUT_IMAGE_DIR):
        os.mkdir(OUTPUT_IMAGE_DIR)
    
    # save region of interest image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_region_of_interest.jpg"), img_reg_of_int)

# visualize region of interest
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test3.jpg"))
ax1.imshow(img, cmap='gray')
ax1.set_title('Original Image', fontsize=30)

img_reg_of_int = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test3_region_of_interest.jpg"))
ax2.imshow(img_reg_of_int, cmap='gray')
ax2.set_title('Region of Interest Image', fontsize=30)

## Apply Perspective Transform ("Birds-Eye View")

In [ ]:
import cv2 
import os
import matplotlib.pyplot as plt
%matplotlib inline

# for each test image apply pipeline steps and stop after perspective transformation applied
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    img = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    img_warped = lane_finder.pipeline(image=img, stop_on_step='warp_perspective')
    
    # create output directory for images, if does not exist
    if not os.path.isdir(OUTPUT_IMAGE_DIR):
        os.mkdir(OUTPUT_IMAGE_DIR)
    
    # save undistorted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_warped.jpg"), img_warped)

# visualize perspective transformation
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

img = plt.imread(os.path.join(TEST_IMAGE_DIR, "test3.jpg"))
ax1.imshow(img, cmap='gray')
ax1.set_title('Original Image', fontsize=30)

img_warped = plt.imread(os.path.join(OUTPUT_IMAGE_DIR, "test3_warped.jpg"))
ax2.imshow(img_warped, cmap='gray')
ax2.set_title('Warped Image', fontsize=30)

## Detect Lane Pixels and Fit to Find Lane Boundary

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# for each test image apply pipeline steps and stop after fit polynomial step applied
test_img_paths = os.listdir(TEST_IMAGE_DIR)

for fname in test_img_paths:
    # the following hack needed to start sliding window search for each new image
    lane_finder._left_line.detected = False
    lane_finder._right_line.detected = False
    
    image = cv2.imread(os.path.join(TEST_IMAGE_DIR, fname))
    image_fit1 = lane_finder.pipeline(image, stop_on_step='fit_polynomial') 
    image_fit2 = lane_finder.pipeline(image, stop_on_step='fit_polynomial')
    
    # create output directory for images, if does not exist
    if not os.path.isdir(OUTPUT_IMAGE_DIR):
        os.mkdir(OUTPUT_IMAGE_DIR)
    
    # save fitted image
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_first_fit_polynomial.jpg"), image_fit1)
    cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, fname[0:-4] + "_second_fit_polynomial.jpg"), image_fit2)

# visualize sliding window search and fitted curve
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

ax1.imshow(cv2.imread(os.path.join(OUTPUT_IMAGE_DIR, 'test3_first_fit_polynomial.jpg')))
ax1.set_title('Sliding Window Search Image', fontsize=30)

ax2.imshow(cv2.imread(os.path.join(OUTPUT_IMAGE_DIR, 'test3_second_fit_polynomial.jpg')))
ax2.set_title('Skip Sliding Window Search Image', fontsize=30)

## Determine Curvature of Lane and Vehicle Position with Respect to Center